In [1]:
import torch 
import pandas as pd
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import sys
sys.path.append('./data_prep')
from sentence_dataset_class import ProcessedSentences
from sentence_processing import build_vocab,sentence_processing
sys.path.append('./transformer_testing')
from tomislav_transformer import Seq2SeqTransformer

/home/tignjatov/anaconda3/envs/ml/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def stringify_series(df):
    df['input_data'] = df['input_data'].astype('string')
    df['output_data'] = df['output_data'].astype('string')
    return df

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
df_train = pd.read_json('data/train_data.json')
df_test = pd.read_json('data/test_data.json')

In [5]:
token_transform = get_tokenizer('basic_english')

In [6]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [7]:
train_input_vocab = build_vocab(df_train['input_data'],token_transform,special_symbols)
train_output_vocab = build_vocab(df_train['output_data'],token_transform,special_symbols)

In [8]:
# train_input_sentences = [sentence_processing(sentence,
#                                              train_input_vocab,
#                                              token_transform,
#                                              special_symbols.index('<bos>'),special_symbols.index('<eos>'))
#                          for sentence in df_train['input_data'].values]
# train_output_sentences = [sentence_processing(sentence,
#                                               train_output_vocab,
#                                               token_transform,
#                                               special_symbols.index('<bos>'),
#                                               special_symbols.index('<eos>'))
#                           for sentence in df_train['output_data'].values]

In [9]:
# test_input_sentences = [sentence_processing(sentence,train_input_vocab,token_transform,special_symbols.index('<bos>'),special_symbols.index('<eos>')) for sentence in df_test['input_data'].values]
# test_output_sentences = [sentence_processing(sentence,train_output_vocab,token_transform,special_symbols.index('<bos>'),special_symbols.index('<eos>')) for sentence in df_test['output_data'].values]

In [10]:
# # train_input_sentences = pad_sequence(train_input_sentences,batch_first=True,padding_value=PAD_IDX)
# # train_output_sentences = pad_sequence(train_output_sentences,batch_first=True,padding_value=PAD_IDX)
# # test_input_sentences = pad_sequence(test_input_sentences,batch_first=True,padding_value=PAD_IDX)
# # test_output_sentences = pad_sequence(test_output_sentences,batch_first=True,padding_value=PAD_IDX)

# train_input_sentences_padded = pad_sequence(train_input_sentences,padding_value=PAD_IDX)
# train_output_sentences_padded = pad_sequence(train_output_sentences,padding_value=PAD_IDX)
# test_input_sentences_padded = pad_sequence(test_input_sentences,padding_value=PAD_IDX)
# test_output_sentences_padded = pad_sequence(test_output_sentences,padding_value=PAD_IDX)

In [8]:
train_dataset = ProcessedSentences(
    input_data = df_train['input_data'].values,
    output_data = df_train['output_data'].values,
)

In [9]:
test_dataset = ProcessedSentences(
    input_data = df_test['input_data'].values,
    output_data = df_test['output_data'].values
)

In [10]:
def collate_fn(batch):
    input_tensor = []
    output_tensor = []
    for input,output in batch:
        input_tensor.append(sentence_processing(input,
                                             train_input_vocab,
                                             token_transform,
                                             special_symbols.index('<bos>'),special_symbols.index('<eos>')
                                             )
                            )
        output_tensor.append(sentence_processing(output,
                                             train_output_vocab,
                                             token_transform,
                                             special_symbols.index('<bos>'),special_symbols.index('<eos>')
                                             ))
    src_batch = pad_sequence(input_tensor, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(output_tensor, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [11]:
train_dataloader = DataLoader(train_dataset,batch_size=128,shuffle=True,collate_fn=collate_fn)
#test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=True)

In [12]:
len(train_dataloader)

1661

In [15]:
torch.manual_seed(0)
input_vocab_size = len(train_input_vocab)
output_vocab_size = len(train_output_vocab)
emb_size = 512
n_head = 8
ffn_hid_dim = 512
batch_size = 128
num_encoder_layers = 3
num_decoder_layers = 3

In [16]:
transformer = Seq2SeqTransformer(
    num_encoder_layers,
    num_decoder_layers,
    emb_size,
    n_head,
    input_vocab_size,
    output_vocab_size,
    ffn_hid_dim)

In [17]:
transformer = transformer.to(device)

In [18]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [19]:
def train_epoch(model,optimizer):
    model.train()
    losses = 0
    train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)
    for input_sent, output_sent in train_dataloader:
        input_sent = input_sent.to(device)
        output_sent = output_sent.to(device)
        
        output_input = output_sent[:-1,:]
        
        input_mask, output_mask, input_padding_mask, output_padding_mask = create_mask(input_sent,output_input)
        logits = model(
            input_sent,
            output_input,
            input_mask,
            output_mask,
            input_padding_mask,
            output_padding_mask,
            input_padding_mask)
        optimizer.zero_grad()
        
        output_out = output_sent[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), output_out.reshape(-1))
        loss.backward()
        
        optimizer.step()
        losses += loss.item()
    return losses/len(train_dataloader)
def evaluate(model):
    model.eval()
    losses = 0
    test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)
    for input_sent, output_sent in test_dataloader:
        input_sent = input_sent.to(device)
        output_sent = output_sent.to(device)
        
        output_input = output_sent[:-1,:]
        input_mask, output_mask, input_padding_mask, output_padding_mask = create_mask(input_sent,output_input)
        logits = model(
            input_sent,
            output_input,
            input_mask,
            output_mask,
            input_padding_mask,
            output_padding_mask,
            input_padding_mask)
        
        
        output_out = output_sent[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), output_out.reshape(-1))
        
        losses += loss.item()
    return losses/len(test_dataloader)
    
    

In [20]:
import tqdm
num_epochs = 10

for epoch in range(1,num_epochs+1):
    train_loss = train_epoch(transformer,optimizer)
    val_loss = evaluate(transformer)
    print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}")

torch.Size([54, 32])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  22,    5,    5,  ..., 2407,   91, 1191],
        [   8,  131,    0,  ...,    6,  571,   14],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')
torch.Size([68, 32])
tensor([[    2,     2,     2,  ...,     2,     2,     2],
        [  192,  2983,     8,  ..., 10198,     9,     0],
        [   17,     7,     5,  ...,  2979,    41,    32],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0')
torch.Size([30, 32])
tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,

RuntimeError: CUDA out of memory. Tried to allocate 300.00 MiB (GPU 0; 3.94 GiB total capacity; 1.89 GiB already allocated; 225.38 MiB free; 2.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
import json

hyper_params = {}
with open('hyperparameters.json', 'r') as f:
    hyper_params = json.load(f)

with open('test.json', 'w' ) as f :
    json.dump(hyper_params, f)

In [17]:
hyper_params

{'emb_size': 256,
 'n_head': 2,
 'ffn_hid_dim': 512,
 'batch_size': 128,
 'num_encoder_layers': 1,
 'num_decoder_layers': 1,
 'epochs': 2}

In [1]:
import time

In [5]:
time.strftime("%d_%m-%Y_%H_%M_%S", time.localtime())

'05_08-2022_01_32_20.json'

In [9]:
from torchtext.data.metrics import bleu_score
import torch

In [11]:
bleu_score(torch.tensor([1,1,1,1,1,1]),torch.tensor([1,1,1,1,1,1]))

TypeError: len() of a 0-d tensor

In [18]:
from torchtext.data.metrics import bleu_score
candidate_corpus = [['My', 'full', 'pytorch', 'test'], ['Another', 'Sentence']]
references_corpus = [[['My', 'full', 'pytorch', 'test'], ['Completely', 'Different']], [['No', 'Match']]]
bleu_score(candidate_corpus, references_corpus)


0.8408964276313782

In [19]:
from torchtext.data.utils import get_tokenizer
tok = get_tokenizer("basic_english")

In [20]:
example_sentences = ['Give me the documents',
                     'Return the files right away',
                     'I need the documents',
                     'I want to return the files',
                     'I want to return the files right away',
                     'Answer the phone right away',
                     ]
my_sentences = ['Please give me the documents',
                'Please return the files right away',
                'Please answer the phone right away',
                'Please return the files',
                'Answer the phone right away',
                'Wake up']

In [29]:
tok_ex = [tok(s )for s in example_sentences]
tok_my = [tok(s) for s in my_sentences]
bleu_score(tok_my,[tok_ex,tok_ex])

AssertionError: The length of candidate and reference corpus should be the same